In [1]:
!wget -O raw_2016.csv -nc "http://www.belpa.be/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN&pin_inprod=1"        

--2018-01-16 18:59:05--  http://www.belpa.be/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN&pin_inprod=1
Resolving www.belpa.be... 195.244.170.200
Connecting to www.belpa.be|195.244.170.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.ms-excel]
Saving to: ‘raw_2016.csv’

raw_2016.csv            [          <=>       ]  47.00M   389KB/s    in 71s     

2018-01-16 19:00:46 (675 KB/s) - ‘raw_2016.csv’ saved [49279680]



In [2]:
import pandas as pd

In [3]:
df_raw = pd.read_csv('raw_2016.csv', delimiter=';', encoding='utf-8')
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,' T KALVERHOF,2360,Oud-Turnhout,"9.922,73","120,21","10.042,94",2015,"9.701,88","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,' T KALVERHOF,2360,Oud-Turnhout,"18.679,77","0,00","18.679,77",2016,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,' T KRISTOFFELHOF,2440,Geel,"902,53","0,00","902,53",2015,"902,53","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,'S JONGERS DIRK,2630,Aartselaar,"0,00","-2.256,06","-2.256,06",2016,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,'S JONGERS RENE,2550,Kontich,"20.412,69","6.079,19","26.491,88",2015,"19.429,46","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [4]:
id_vars = 'name	postal_code	municipality	eagf	eafr	total	year'.split('\t')
all_columns = list(df_raw.columns)
total_cols = ['total', 'eafr', 'eagf']
value_vars = list(set(all_columns) - set(id_vars))

In [5]:
def fix_nums(series):
    return series.astype(str).str.replace('.', '').str.replace(',', '.')

for val in value_vars + total_cols:
    df_raw[val] = pd.to_numeric(fix_nums(df_raw[val]), errors='coerce')
    
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,' T KALVERHOF,2360,Oud-Turnhout,9922.73,120.21,10042.94,2015,9701.88,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,' T KALVERHOF,2360,Oud-Turnhout,18679.77,0.00,18679.77,2016,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,' T KRISTOFFELHOF,2440,Geel,902.53,0.00,902.53,2015,902.53,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,'S JONGERS DIRK,2630,Aartselaar,0.00,-2256.06,-2256.06,2016,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,'S JONGERS RENE,2550,Kontich,20412.69,6079.19,26491.88,2015,19429.46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, value_name='amount', var_name='scheme')
df = df[df['amount'] > 0]
df = df.drop(total_cols, 1)
df.head()

,name,postal_code,municipality,year,scheme,amount
84214,ADMINISTRATION COMMUNALE DE CHIEVRES,7950,Chièvres,2016,V/B.3.2,304949.98
98593,BELVAS SPRL,7822,Ath,2015,V/B.3.2,152896.68
101291,BRASSERIE COOPERATIVE DE LA LESSE SCRL,5580,Rochefort,2015,V/B.3.2,12410.00
101292,BRASSERIE D ECAUSSINNES SA,7190,Ecaussinnes,2015,V/B.3.2,37418.15
101293,BRASSERIE DES 3 F SPRL,7522,Tournai,2016,V/B.3.2,368900.00


In [7]:
df = df.rename(columns={
    'name': 'recipient_name',
    'postal_code': 'recipient_postcode',
    'municipality': 'recipient_location',
})
df['currency'] = 'EUR'
df['country'] = 'BE'

def generate_id(row):
    return u'BE-%s-%s' % (row['recipient_postcode'], row['recipient_name'])

df['recipient_id'] = df.apply(generate_id, 1)
df.head()

,recipient_name,recipient_postcode,recipient_location,year,scheme,amount,currency,country,recipient_id
84214,ADMINISTRATION COMMUNALE DE CHIEVRES,7950,Chièvres,2016,V/B.3.2,304949.98,EUR,BE,BE-7950-ADMINISTRATION COMMUNALE DE CHIEVRES
98593,BELVAS SPRL,7822,Ath,2015,V/B.3.2,152896.68,EUR,BE,BE-7822-BELVAS SPRL
101291,BRASSERIE COOPERATIVE DE LA LESSE SCRL,5580,Rochefort,2015,V/B.3.2,12410.00,EUR,BE,BE-5580-BRASSERIE COOPERATIVE DE LA LESSE SCRL
101292,BRASSERIE D ECAUSSINNES SA,7190,Ecaussinnes,2015,V/B.3.2,37418.15,EUR,BE,BE-7190-BRASSERIE D ECAUSSINNES SA
101293,BRASSERIE DES 3 F SPRL,7522,Tournai,2016,V/B.3.2,368900.00,EUR,BE,BE-7522-BRASSERIE DES 3 F SPRL


In [8]:
df.year.value_counts()

2016    160054
2015    123282
Name: year, dtype: int64

In [9]:
df[df['year'] == 2016].to_csv('be_2016.csv.gz', index=False, encoding='utf-8', compression='gzip')